<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
<center>Автор материала: программист-исследователь Mail.Ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ [Юрий Кашницкий](https://yorko.github.io/)

# <center> Capstone проект №1 <br> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 5.  Соревнование Kaggle "Catch Me If You Can"

На этой неделе мы вспомним про концепцию стохастического градиентного спуска и опробуем классификатор Scikit-learn SGDClassifier, который работает намного быстрее на больших выборках, чем алгоритмы, которые мы тестировали на 4 неделе. Также мы познакомимся с данными [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) Kaggle по идентификации пользователей и сделаем в нем первые посылки. По итогам этой недели дополнительные баллы получат те, кто попадет в топ-30 публичного лидерборда соревнования.

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
**Также рекомендуется вернуться и просмотреть [задание](https://www.coursera.org/learn/supervised-learning/programming/t2Idc/linieinaia-rieghriessiia-i-stokhastichieskii-ghradiientnyi-spusk) "Линейная регрессия и стохастический градиентный спуск" 1 недели 2 курса специализации.**

### Задание
1. Заполните код в этой тетрадке 
2. Если вы проходите специализацию Яндеса и МФТИ, пошлите тетрадку в соответствующем Peer Review. <br> Если вы проходите курс ODS, выберите ответы в [веб-форме](https://docs.google.com/forms/d/1pLsegkAICL9PzOLyAeH9DmDOBfktte0l8JW75uWcTng). 

In [18]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
from tqdm import tqdm
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score

**Считаем данные [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [19]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = 'capstone_user_identification'

In [20]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')

**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [21]:
train_test_df = pd.concat([train_df, test_df], ignore_index=True)

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

In [22]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 21 columns):
site1     253561 non-null int64
time1     253561 non-null object
site2     250098 non-null float64
time2     250098 non-null object
site3     246919 non-null float64
time3     246919 non-null object
site4     244321 non-null float64
time4     244321 non-null object
site5     241829 non-null float64
time5     241829 non-null object
site6     239495 non-null float64
time6     239495 non-null object
site7     237297 non-null float64
time7     237297 non-null object
site8     235224 non-null float64
time8     235224 non-null object
site9     233084 non-null float64
time9     233084 non-null object
site10    231052 non-null float64
time10    231052 non-null object
target    253561 non-null int64
dtypes: float64(9), int64(2), object(10)
memory usage: 42.6+ MB


In [23]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [24]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 20 columns):
site1     82797 non-null int64
time1     82797 non-null object
site2     81308 non-null float64
time2     81308 non-null object
site3     80075 non-null float64
time3     80075 non-null object
site4     79182 non-null float64
time4     79182 non-null object
site5     78341 non-null float64
time5     78341 non-null object
site6     77566 non-null float64
time6     77566 non-null object
site7     76840 non-null float64
time7     76840 non-null object
site8     76151 non-null float64
time8     76151 non-null object
site9     75484 non-null float64
time9     75484 non-null object
site10    74806 non-null float64
time10    74806 non-null object
dtypes: float64(9), int64(1), object(10)
memory usage: 13.3+ MB


**В обучающей выборке – 2297 сессий одного пользователя (Alice) и 251264 сессий – других пользователей, не Элис. Дисбаланс классов очень сильный, и смотреть на долю верных ответов (accuracy) непоказательно.**

In [25]:
train_df['target'].value_counts()

0    251264
1      2297
Name: target, dtype: int64

**Пока для прогноза будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [26]:
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')

In [27]:
train_test_df_sites.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
0,718,0,0,0,0,0,0,0,0,0
1,890,941,3847,941,942,3846,3847,3846,1516,1518
2,14769,39,14768,14769,37,39,14768,14768,14768,14768
3,782,782,782,782,782,782,782,782,782,782
4,22,177,175,178,177,178,175,177,177,178
5,570,21,570,21,21,0,0,0,0,0
6,803,23,5956,17513,37,21,803,17514,17514,17514
7,22,21,29,5041,14422,23,21,5041,14421,14421
8,668,940,942,941,941,942,940,23,21,22
9,3700,229,570,21,229,21,21,21,2336,2044


**Создайте разреженные матрицы *X_train_sparse* и *X_test_sparse* аналогично тому, как мы это делали ранее. Используйте объединенную матрицу *train_test_df_sites*, потом разделите обратно на обучающую и тестовую части.**

Обратите внимание на то, что в  сессиях меньше 10 сайтов  у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.

**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

In [28]:
def dropcols_coo(M, idx_to_drop):
    idx_to_drop = np.unique(idx_to_drop)
    C = M.tocoo()
    keep = ~np.in1d(C.col, idx_to_drop)
    C.data, C.row, C.col = C.data[keep], C.row[keep], C.col[keep]
    C.col -= idx_to_drop.searchsorted(C.col) # decrement column indices
    C._shape = (C.shape[0], C.shape[1] - len(idx_to_drop))
    return C.tocsr()

In [29]:
def transform_to_csr_matrix(arr):
    rows = []
    columns = []
    data = []
    for i in tqdm(range(arr.shape[0])):
        # Create dictionary {site: number of visits per session}
        unique, counts = np.unique(arr[i], return_counts=True)
        dic = dict(zip(unique, counts))
        rows.extend([i]*len(dic.keys()))
        columns.extend(dic.keys())
        data.extend(dic.values())
    
    # Sparse coo matrix
    arr_new_coo = csr_matrix((data, (rows, columns)))
    
    # Drop column with "zero" site and transform to csr
    return dropcols_coo(arr_new_coo, 0)

In [30]:
def sparse(d):
    df = pd.DataFrame(d)
    row = []
    data = []
    col = []
    for k in tqdm(range(len(df))):
        a = list(df.loc[k+1])
        for i in set(a):
            row.append(k)
            col.append(i)
            data.append(a.count(i))
    c = csr_matrix((data, (row, col)))
    return dropcols_coo(c, 0)

In [31]:
train_test_sparse = transform_to_csr_matrix(train_test_df_sites.values)
X_train_sparse = train_test_sparse[:253561]
X_test_sparse = train_test_sparse[253561:]
y = train_df['target']

100%|██████████| 336358/336358 [00:05<00:00, 58082.27it/s]


**<font color='red'>Вопрос 1. </font> Выведите размерности матриц *X_train_sparse* и *X_test_sparse* – 4 числа на одной строке через пробел: число строк и столбцов матрицы *X_train_sparse*, затем число строк и столбцов матрицы *X_test_sparse*.**

In [32]:
list(X_train_sparse.shape)+list(X_test_sparse.shape)

[253561, 48371, 82797, 48371]

In [33]:
y.shape

(253561,)

In [34]:
with open(os.path.join(PATH_TO_DATA, 'site_dic.pkl'), 'rb') as s:
        d = pickle.load(s)

In [35]:
features = ['unique', 'site_max_count', 'com_count', 'fr_count',
            'org_count', 'net_count', 'hour', 'dayofweek', 'month', 'year', 'weekend']

Создание признаков:
- unique - количество уникальных сайтов за сессию
- site_max_count - сайт наиболее часто встречающийся за сессию (если таких несколько, то первый из них)
- com_count - количество сайтов за сессию с расширением .com
- org_count - количество сайтов за сессию с расширением .org
- fr_count - количество сайтов за сессию с расширением .fr
- net_count - количество сайтов за сессию с расширением .net
- hour - время начала сессии (утро, день, вечер, ночь)
- dayofwee - день недели начала сессии
- month - месяц начала сессии
- year - год начала сессии
- weekend - относится ли день начала сессии к выходному дню

In [36]:
site_max_count=[]
unique=[]
for k in tqdm(range(len(train_test_df_sites.values))):  
    s = list(train_test_df_sites.values[k])
    t=set(s)
    t-={0}
    unique.append(len(t))
    m = None
    q_max = 0
    for item in set(s):
        q = s.count(item)
        if q > q_max:
            q_max = q
            m = item
    site_max_count.append(m)

100%|██████████| 336358/336358 [00:03<00:00, 107668.37it/s]


In [37]:
t = pd.to_datetime(train_test_df['time1'])
hour=[]
for i in tqdm(range(len(t))):
    if ((t[i].hour >= 6) & (t[i].hour <= 11)):
        hour.append(1)
    else:
        if ((t[i].hour >= 12) & (t[i].hour <= 18)):
            hour.append(2)
        else:
            if ((t[i].hour >= 19) & (t[i].hour <= 23)):
                hour.append(3)
            else:
                hour.append(4)
                    

100%|██████████| 336358/336358 [00:10<00:00, 33283.18it/s]


In [38]:
dayofweek=[]
month=[]
year=[]
for i in tqdm(range(len(t))):
    dayofweek.append(t[i].dayofweek)
    month.append(t[i].month)
    year.append(t[i].year)


100%|██████████| 336358/336358 [00:10<00:00, 32258.33it/s]


In [39]:
weekend=[]
for i in dayofweek:
    if i >5:
        weekend.append(1)
    else:
        weekend.append(0)

In [40]:
tts = train_test_df_sites.values
d1 = {v:k for k, v in d.items()}
d1[0]=0

In [41]:
r = []  
for k in tqdm(range(len(tts))):  
    for i in range(10):
        if d1[tts[k][i]] != 0:
              r.append(d1[tts[k][i]].split('.')[-1])
        else:
              r.append(0)    

100%|██████████| 336358/336358 [00:04<00:00, 78554.32it/s]


In [42]:
r = np.array(r).reshape((336358, 10))

In [43]:
com_count = []
fr_count = []
org_count = []
net_count = []
for i in tqdm(range(len(r))):
    com_count.append(list(r[i]).count('com'))
    fr_count.append(list(r[i]).count('fr'))
    org_count.append(list(r[i]).count('org'))
    net_count.append(list(r[i]).count('net'))


100%|██████████| 336358/336358 [00:04<00:00, 79877.30it/s]


In [44]:
train_test_df_sites['com_count']=com_count
train_test_df_sites['fr_count']=fr_count
train_test_df_sites['org_count']=org_count
train_test_df_sites['net_count']=net_count
train_test_df_sites['unique']=unique
train_test_df_sites['site_max_count']=site_max_count
train_test_df_sites['hour']=hour
train_test_df_sites['dayofweek']=dayofweek
train_test_df_sites['month']=month
train_test_df_sites['year']=year
train_test_df_sites['weekend']=weekend


**Сохраним в pickle-файлы объекты *X_train_sparse*, *X_test_sparse* и *y* (последний – в файл *kaggle_data/train_target.pkl*).**

In [45]:
with open(os.path.join(PATH_TO_DATA, 'X_train_sparse.pkl'), 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_test_sparse.pkl'), 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'train_target.pkl'), 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl, protocol=2)

**Разобьем обучающую выборку на 2 части в пропорции 7/3, причем не перемешивая. Исходные данные упорядочены по времени, тестовая выборка по времени четко отделена от обучающей, это же соблюдем и здесь.**

In [46]:
train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

**Создайте объект `sklearn.linear_model.SGDClassifier` с логистической функцией потерь и параметром *random_state*=17. Остальные параметры оставьте по умолчанию, разве что *n_jobs*=-1 никогда не помешает. Обучите  модель на выборке `(X_train, y_train)`.**

In [47]:
sgd_logit_base = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit_base.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=17, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [48]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [49]:
def sgd_logit(train_test_sparse):
    X_train_sparse = train_test_sparse[:253561]
    X_test_sparse = train_test_sparse[253561:]
    y = train_df['target']
    train_share = int(.7 * X_train_sparse.shape[0])
    X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
    X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]
    sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
    sgd_logit.fit(X_train, y_train)
    logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)
    print(round(roc_auc_score(y_valid, logit_valid_pred_proba[:,1]), 3))

In [50]:
def logit(train_test_sparse):
    X_train_sparse = train_test_sparse[:253561]
    X_test_sparse = train_test_sparse[253561:]
    y = train_df['target']
    train_share = int(.7 * X_train_sparse.shape[0])
    X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
    X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]
    logit = LogisticRegression(random_state=17, n_jobs=-1)
    logit.fit(X_train, y_train)
    logit_valid_pred_proba = logit.predict_proba(X_valid)
    print(round(roc_auc_score(y_valid, logit_valid_pred_proba[:,1]), 3))

In [51]:
sgd_logit(train_test_sparse=train_test_sparse)

0.934


In [52]:
logit(train_test_sparse=train_test_sparse)

0.958


In [53]:
from scipy.sparse import hstack

In [54]:
for i in features:
    train_test_sparse1 = hstack((train_test_sparse, np.array(train_test_df_sites[i])[:,None]), format= 'csr')
    print(i)
    sgd_logit(train_test_sparse=train_test_sparse1)

unique
0.921
site_max_count
0.474
com_count
0.921
fr_count
0.918
org_count
0.933
net_count
0.932
hour
0.899
dayofweek
0.925
month
0.922
year
0.5
weekend
0.935


In [55]:
for i in features:
    train_test_sparse1 = hstack((train_test_sparse, np.array(train_test_df_sites[i])[:,None]), format= 'csr')
    print(i)
    logit(train_test_sparse=train_test_sparse1)

unique
0.958
site_max_count
0.953
com_count
0.958
fr_count
0.958
org_count
0.958
net_count
0.958
hour
0.969
dayofweek
0.96
month
0.962
year
0.95
weekend
0.959


Построение модели с признаками: weekend, hour, dayofweek, org_count, com_count

In [56]:
train_test_sparse1 = hstack((train_test_sparse,np.array(train_test_df_sites['weekend'])[:,None], np.array(train_test_df_sites['hour'])[:,None], np.array(train_test_df_sites['dayofweek'])[:,None], np.array(train_test_df_sites['org_count'])[:,None],np.array(train_test_df_sites['com_count'])[:,None]), format= 'csr')
logit(train_test_sparse=train_test_sparse1)

0.971


In [57]:
X_train_sparse = train_test_sparse1[:253561]

In [58]:
c_values = np.logspace(-2, 2, 10)

logit1 = LogisticRegressionCV(Cs= c_values, random_state=17, n_jobs=-1, scoring='roc_auc')
logit1.fit(X_train_sparse, y)

LogisticRegressionCV(Cs=array([1.00000e-02, 2.78256e-02, 7.74264e-02, 2.15443e-01, 5.99484e-01,
       1.66810e+00, 4.64159e+00, 1.29155e+01, 3.59381e+01, 1.00000e+02]),
           class_weight=None, cv='warn', dual=False, fit_intercept=True,
           intercept_scaling=1.0, max_iter=100, multi_class='warn',
           n_jobs=-1, penalty='l2', random_state=17, refit=True,
           scoring='roc_auc', solver='lbfgs', tol=0.0001, verbose=0)

**Сделайте прогноз в виде предсказанных вероятностей того, что это сессия Элис, на отложенной выборке *(X_valid, y_valid)*.**

In [59]:
logit_valid_pred_proba = sgd_logit_base.predict_proba(X_valid)

**<font color='red'>Вопрос 2. </font> Посчитайте ROC AUC логистической регрессии, обученной с помощью стохастического градиентного спуска, на отложенной выборке. Округлите до 3 знаков после разделителя.**

In [60]:
round(roc_auc_score(y_valid, logit_valid_pred_proba[:,1]), 3)

0.934

**Сделайте прогноз в виде предсказанных вероятностей отнесения к классу 1 для тестовой выборки с помощью той же *sgd_logit*, обученной уже на всей обучающей выборке (а не на 70%).**

In [61]:
%%time
sgd_logit_base.fit(X_train_sparse, y)
logit_test_pred_proba = sgd_logit_base.predict_proba(X_test_sparse)

ValueError: X has 48371 features per sample; expecting 48376

In [62]:
X_test_sparse = train_test_sparse1[253561:]

In [63]:
logit_test_pred = logit1.predict_proba(X_test_sparse)

**Запишите ответы в файл и сделайте посылку на Kaggle. Дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] Coursera_Username", чтоб можно было легко идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard/public).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить.**

In [66]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [67]:
write_to_submission_file(logit_test_pred[:,1], 'pred.csv')

Результат, полученный на лидерборде, равен 0.93060

## Критерии оценки работы (только для Peer Review в специализации):
- Правильные ли получились размерности матриц в п. 1? (max. 2 балла)
- Правильным ли получилось значения ROC AUC в п. 2? (max. 4 балла)
- Побит ли бенчмарк "sgd_logit_benchmark.csv" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)
- Побит ли бенчмарк "Logit +3 features" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)

## Пути улучшения
На этой неделе дается много времени на соревнование. Не забывайте вносить хорошие идеи, к которым Вы пришли по ходу соревнования, в описание финального проекта (`html`, `pdf` или `ipynb`). Это только в случае, если вы проходите специализацию.
Что можно попробовать:
 - Использовать ранее построенные признаки для улучшения модели (проверить их можно на меньшей выборке по 150 пользователям, отделив одного из пользователей от остальных – это быстрее)
 - Настроить параметры моделей (например, коэффициенты регуляризации)
 - Если позволяют мощности (или хватает терпения), можно попробовать смешивание (блендинг) ответов бустинга и линейной модели. [Вот](http://mlwave.com/kaggle-ensembling-guide/) один из самых известных тьюториалов по смешиванию ответов алгоритмов, также хороша [статья](https://alexanderdyakonov.wordpress.com/2017/03/10/cтекинг-stacking-и-блендинг-blending) Александра Дьяконова
 - Обратите внимание, что в соревновании также даны исходные данные о посещенных веб-страницах Элис и остальными 1557 пользователями (*train.zip*). По этим данным можно сформировать свою обучающую выборку. 

На 6 неделе мы пройдем большой тьюториал по Vowpal Wabbit и попробуем его в деле, на данных соревнования.